# Importing Libraries and Data

In [1]:
import pandas as pd

In [4]:
advanced = pd.read_csv('./data/advanced', index_col=0)
touches = pd.read_csv('./data/touches', index_col=0)
drives = pd.read_csv('./data/drives', index_col=0)
defensive_impact = pd.read_csv('./data/defensive_impact', index_col=0)
passing = pd.read_csv('./data/passing')
shooting_efficiency = pd.read_csv('./data/shooting_efficiency', index_col=0)
speed_distance = pd.read_csv('./data/speed_distance', index_col=0)
rebounding = pd.read_csv('./data/rebounding', index_col=0)
catch_shoot = pd.read_csv('./data/catch_shoot')
pullup_shooting = pd.read_csv('./data/pullup_shooting', index_col=0)
elbow_touches = pd.read_csv('./data/elbow_touches', index_col=0)
post_ups = pd.read_csv('./data/post_ups', index_col=0)
paint_touches = pd.read_csv('./data/paint_touches', index_col=0)
offensive_rebounding = pd.read_csv('./data/offensive_rebounding', index_col=0)
defensive_rebounding = pd.read_csv('./data/defensive_rebounding', index_col=0)
hustle = pd.read_csv('./data/hustle', index_col=0)

# Looking Over the Potential Predicted Stats

In [14]:
# Making it possible to fully view the dataframes
pd.options.display.max_columns = 25

### 'Action' Stats

In [18]:
drives.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,GP,W,L,MIN,DRIVES,DRIVE_FGM,DRIVE_FGA,DRIVE_FG_PCT,DRIVE_FTM,DRIVE_FTA,DRIVE_FT_PCT,DRIVE_PTS,DRIVE_PTS_PCT,DRIVE_PASSES,DRIVE_PASSES_PCT,DRIVE_AST,DRIVE_AST_PCT,DRIVE_TOV,DRIVE_TOV_PCT,DRIVE_PF,DRIVE_PF_PCT
0,201166,Aaron Brooks,1610612750,MIN,32,19,13,5.9,1.8,0.4,0.9,0.400,0.1,0.1,0.750,0.8,0.482,0.6,0.321,0.1,0.054,0.0,0.018,0.1,0.036
1,203932,Aaron Gordon,1610612753,ORL,58,19,39,32.9,5.2,0.9,2.5,0.379,0.7,1.0,0.700,2.8,0.538,1.2,0.233,0.3,0.066,0.5,0.093,0.5,0.100
2,1626151,Aaron Harrison,1610612742,DAL,9,2,7,25.9,2.4,0.4,1.2,0.364,0.0,0.0,0.000,0.9,0.364,0.7,0.273,0.1,0.045,0.0,0.000,0.0,0.000
3,1627846,Abdel Nader,1610612738,BOS,47,32,15,11.1,2.2,0.3,1.0,0.304,0.1,0.1,0.667,0.7,0.311,0.6,0.291,0.2,0.078,0.3,0.117,0.1,0.029
4,203940,Adreian Payne,1610612753,ORL,5,0,5,8.5,0.2,0.2,0.2,1.000,0.0,0.0,0.000,0.4,2.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000


In [17]:
drives['DRIVE_FGM']*drives['DRIVE_FG_PCT']/drives['DRIVE_FGA'] + (
    drives['DRIVE_FTM']*drives['DRIVE_FT_PCT']/drives['DRIVE_FTA']) + (
    drives['DRIVE_FGM']*drives['DRIVE_FG_PCT']/drives['DRIVE_FGA']
)

SyntaxError: invalid syntax (<ipython-input-17-a781238e4445>, line 1)

In [23]:
(drives['DRIVE_PTS']/drives['DRIVES'] - drives['DRIVE_PTS_PCT']).mean()

0.003926699307122114

### Touches

What do I need

- Drives (rate compared to minutes or posessions, ppp (assists*avg points from assists + actual points - turnovers*avg points from turnovers)

GET THIS FROM BBALL REFERENCE

For assists (3s from AST * 3 + 2s from AST * 2 /AST)

3FG% * 3FGA% * 3FG%AST = 3s from AST
2FG% * 2FGA% * 2FG%AST = 2s from AST
ASTs = 3's from AST + 2s from AST


FROM NBA.COM TEAM STATS GENERAL -> MISC and from GENERAL -> TRADITIONAL
Sum(Pts Off TO) / Sum(TO)



In [26]:
(shooting_efficiency['POINTS'] - (shooting_efficiency['DRIVE_PTS'] + shooting_efficiency['CATCH_SHOOT_PTS'] + shooting_efficiency['PULL_UP_PTS'] + shooting_efficiency['PAINT_TOUCH_PTS'] + shooting_efficiency['POST_TOUCH_PTS'] + shooting_efficiency['ELBOW_TOUCH_PTS'])).mean()

0.5211313131313141